In [1]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import numpy as np
import onnxruntime as ort
folder = './musicgen-small'

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
processor = AutoProcessor.from_pretrained("facebook/musicgen-stereo-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth asd asd asd"],
    padding=True,
    return_tensors="pt",
)

In [4]:
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [5]:
res = model.generate(**inputs, do_sample=True, guidance_scale=5, max_new_tokens=256) # , temperature=0.7, top_k=250, top_p=1.0

((tensor([[[[ 0.4299,  0.2789, -0.9029,  ...,  0.4368,  0.2274, -1.7982]],

         [[-1.2214,  0.2943,  1.4827,  ...,  0.7709,  0.3465,  0.0166]],

         [[ 0.4315,  1.2502,  1.3468,  ..., -0.6463,  0.4384, -0.3837]],

         ...,

         [[ 0.2730,  0.7049,  0.3404,  ...,  0.3137, -0.1228, -0.0933]],

         [[-0.8339,  1.1280, -1.8324,  ...,  1.1933,  2.9696, -1.6479]],

         [[ 0.1308, -1.2332, -0.9648,  ..., -1.4705,  2.4805, -0.9702]]],


        [[[ 0.4299,  0.2789, -0.9029,  ...,  0.4368,  0.2274, -1.7982]],

         [[-1.2214,  0.2943,  1.4827,  ...,  0.7709,  0.3465,  0.0166]],

         [[ 0.4315,  1.2502,  1.3468,  ..., -0.6463,  0.4384, -0.3837]],

         ...,

         [[ 0.2730,  0.7049,  0.3404,  ...,  0.3137, -0.1228, -0.0933]],

         [[-0.8339,  1.1280, -1.8324,  ...,  1.1933,  2.9696, -1.6479]],

         [[ 0.1308, -1.2332, -0.9648,  ..., -1.4705,  2.4805, -0.9702]]]]), tensor([[[[ 3.4204e-02,  5.8930e-01, -3.6368e-04,  ...,  5.9485e-04,
       

RuntimeError: No active exception to reraise

In [ ]:
raise

tensor([[2048, 1195, 1057,  ...,  791, 1327, 1549],
        [2048, 1534, 1457,  ...,  727,  597, 2018],
        [2048,  741, 1421,  ..., 1646,  214, 1534],
        [2048,   20,  475,  ..., 1930, 1573,  824]])

In [ ]:
from IPython.display import Audio

sampling_rate = 32000
Audio(res[0][0].detach().numpy(), rate=sampling_rate)

In [4]:
text_encoder_session = ort.InferenceSession(f"{folder}/text_encoder.onnx")
decoder_model_merged_session = ort.InferenceSession(f"{folder}/decoder_model_merged.onnx")
audio_decoder = ort.InferenceSession(f"{folder}/encodec_decode.onnx")
delay_pattern_mask_builder = ort.InferenceSession(f"{folder}/build_delay_pattern_mask.onnx")

encoded = text_encoder_session.run(None, {"input_ids": inputs['input_ids'].detach().numpy(), 'attention_mask': inputs['attention_mask'].detach().numpy()})[0]
decoder_inputs = {
    'encoder_attention_mask': inputs['attention_mask'].detach().numpy(),
    'input_ids': np.array([[2048],[2048],[2048],[2048]], dtype=np.int64),
    'encoder_hidden_states': encoded,
    'use_cache_branch': np.array([True])
}
for i in range(24):
    key_ = f"past_key_values.{i}.decoder.key"
    decoder_inputs.update({key_: np.random.randn(1,16,1,64).astype(np.float32)})
    key_ = f"past_key_values.{i}.decoder.value"
    decoder_inputs.update({key_: np.random.randn(1,16,1,64).astype(np.float32)})
    key_ = f"past_key_values.{i}.encoder.key"
    decoder_inputs.update({key_: np.random.randn(1,16,18,64).astype(np.float32)})
    key_ = f"past_key_values.{i}.encoder.value"
    decoder_inputs.update({key_: np.random.randn(1,16,18,64).astype(np.float32)})

for i in range(256-1):
    print(i, end='\r')
    decoder_output = decoder_model_merged_session.run(None, decoder_inputs)
    logits = decoder_output[0][:, -1][:, None]
    kv = decoder_output[1:]
    decoder_inputs['input_ids'] = np.concatenate([decoder_inputs['input_ids'], np.argmax(logits, axis=-1)], axis=-1)
    if i == 0:
        for j in range(24):
            kvs = kv[j * 4 : (j + 1) * 4]
            key_ = f"past_key_values.{j}.decoder.key"
            decoder_inputs[key_] = kvs[0][:,:,-1][:,:,None] # np.concatenate([decoder_inputs[key_], kvs[0][:,:,-1][:,:,None,:]], axis=2)
            print(kvs[0][:,:,-1][:,:,None].shape)
            key_ = f"past_key_values.{j}.decoder.value"
            decoder_inputs[key_] = kvs[1][:,:,-1][:,:,None] # np.concatenate([decoder_inputs[key_], kvs[1][:,:,-1][:,:,None,:]], axis=2)
    else:
        for j in range(24):
            kvs = kv[j * 4 : (j + 1) * 4]
            key_ = f"past_key_values.{j}.decoder.key"
            decoder_inputs[key_] = kvs[0][:,:,-1][:,:,None] # np.concatenate([decoder_inputs[key_], kvs[0][:,:,-1][:,:,None,:]], axis=2)
            print(kvs[0].shape)
            key_ = f"past_key_values.{j}.decoder.value"
            decoder_inputs[key_] = kvs[1][:,:,-1][:,:,None] # np.concatenate([decoder_inputs[key_], kvs[1][:,:,-1][:,:,None,:]], axis=2)

(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 1, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 3, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4, 64)
(1, 16, 4,

In [5]:
delay_pattern_inputs = {
    'input_ids': np.array([[-1,-1,-1,-1]]*16, dtype=np.int64).swapaxes(0,1),
    'pad_token_id': np.array([2048], dtype=np.int64),
    'max_length': np.array([256], dtype=np.int64)
}

input_ids, delay_pattern_mask = delay_pattern_mask_builder.run(None, delay_pattern_inputs)
delay_pattern_mask

2024-10-15 01:13:56.192738563 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,16} does not match actual shape of {4,1} for output input_ids_edited


array([[2048,   -1,   -1, ..., 2048, 2048, 2048],
       [2048, 2048,   -1, ...,   -1, 2048, 2048],
       [2048, 2048, 2048, ...,   -1,   -1, 2048],
       [2048, 2048, 2048, ...,   -1,   -1,   -1]], dtype=int64)

In [9]:
output_ids = decoder_inputs['input_ids'][decoder_inputs['input_ids'] != 2048].reshape(
    1, 4, -1
)

In [11]:
output_ids = np.where(delay_pattern_mask == -1, decoder_inputs['input_ids'], delay_pattern_mask)
output_ids = output_ids[output_ids != 2048].reshape(
    1, 4, -1
)
output_ids = output_ids[None, ...]

In [12]:
audio = audio_decoder.run(None, {'audio_codes': output_ids})[0]

In [13]:
from IPython.display import Audio

sampling_rate = 32000
Audio(audio[0], rate=sampling_rate)